In [45]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict
import os
import uuid
from datetime import datetime
import dateutil.parser as parser

In [46]:
etl = 'cuartero2newaqm'

input_schema_file = ''
input_data_file = ''
mapping_file = ''

#datelog_dir = 'couchbase-curis-2019-06-21-cuartero'
datelog_dir = 'couchbase-curis-2019-06-21-pototan'

if etl == 'cuartero2newaqm':
    schema_meta_file = '_meta.csv'
    schema_meta_dir = 'data/processed/' + datelog_dir + '/'
    schema_meta_path = schema_meta_dir + schema_meta_file
    
    #mapping_file = '2.1.TestAQMHealthInfoQuestions.V1.map.csv'
    #mapping_file = '1.1.AQMPersonalQuestions.V1.map.csv'
    #mapping_file = '2.1.AQMGeneralQuestions.V1.map.csv'
    #mapping_file = '2.2.AQMHealthInfoQuestions.V1.map.csv'
    #mapping_file = '2.3.AQMHouseholdQuestions.V1.map.csv'
    #mapping_file = '2.6.AQMMentalHealthQuestions.V1.map.csv'
    mapping_file = '2.7.AWHDisabilityQuestions.V1.map.csv'
    
    mapping_dir = 'schema/map/Philippines/' 
    mapping_path = mapping_dir + mapping_file
    
    #TODO: add cleaned_dir !!!
    tmp_dir = 'data/processed/' + datelog_dir + '/tmp/'
    processed_dir = 'data/cleaned/' + datelog_dir + '/'
    #processed_dir = 'data/processed/' + datelog_dir + '/'
    merged_dir = 'data/merged/' + datelog_dir + '/'
    
    

## TODO: Remove repeatinng values in tolist()

## TODO: create a directory for cleaned file

## TODO: DISABILTY CAUSE MUST BE ARRAY NOT STRING!!

## TODO: CREATE FOLDER DIRECTORY MKDIR

## TODO: FIX ID MERGING

## TODO: FIX _id into profileId

# READ MAPPING FILE

In [47]:
#def _get_mapping_fields():
_mapping_df = pd.DataFrame()
_mapping_df = read_csv(mapping_path).sort_values(['source_key']).replace(np.nan,'',regex=True)
_mapping_df

,source_key,source_type,destination_key,destination_type,data_type,data_source,data_format,data_transformation,default_value,lookup_value
1,mental_healths.cause,string,answers.mentalHealthEffect,string,existing,original,,not required,,
2,mental_healths.experience,string,answers.mentalHealthExperience,string,existing,original,,not required,,
0,organization,string,organization,string,existing,original,,not required,,
3,registered_at,timestamp,dateCreated,timestamp,existing,original,timestamp,required,,
4,user-cam.id,string,createdBy,string,existing,original,,not required,,
5,,,formId,string,new,generated,,not required,ajWmReH7k2G4mVuvYgYrpX,
6,,,formName,string,new,generated,,not required,2.6 - AQM Mental Health Questions V1,
7,,,type,string,new,generated,,not required,profile-related-form,


# GET THE FIELDS IN THE MAPPING FILE

In [48]:
mapping_fields_list = []
mapping_fields_list = list(filter(None, (_mapping_df['source_key'].unique())))
mapping_fields_list

['mental_healths.cause',
 'mental_healths.experience',
 'organization',
 'registered_at',
 'user-cam.id']

# GET THE FIELDS IN THE SCHEMA META (i.e. _meta) FILE

In [49]:
meta_headers_df = pd.DataFrame()
meta_headers_df = pd.read_csv(schema_meta_path)

In [50]:
meta_headers_df

,file_name,field_name,field_type
0,resident,middle_name,primitive
1,resident,birthdate,primitive
2,resident,id,primitive
3,resident,type,primitive
4,resident,first_name,primitive
5,resident,last_name,primitive
6,resident,last_name_suffix,primitive
7,resident,organization,primitive
8,resident,nhid,primitive
9,resident,gender,primitive


## MATCH THE FIELDS IN MAPPING FILE AND SCHEMA META FILE

In [51]:
match_headers_df = pd.DataFrame()
match_headers_df = meta_headers_df[meta_headers_df['field_name'].isin(mapping_fields_list)]
match_headers_df = match_headers_df.sort_values(['file_name','field_name']).reset_index(drop=True)
match_headers_df

,file_name,field_name,field_type
0,mental_healths.cause,mental_healths.cause,list
1,mental_healths.experience,mental_healths.experience,list
2,resident,organization,primitive
3,resident,registered_at,primitive
4,resident,user-cam.id,primitive


# CREATE DEFAULT DICT FOR FILENAME AS KEY and FIELD NAMES AS VALUE

In [52]:
filename_per_field_dd = defaultdict(list)

for index,row in match_headers_df.iterrows():
    filename = row['file_name']
    fields = row['field_name']
    filename_per_field_dd[filename].append(fields)  
    
filename_per_field_dd

defaultdict(list,
            {'mental_healths.cause': ['mental_healths.cause'],
             'mental_healths.experience': ['mental_healths.experience'],
             'resident': ['organization', 'registered_at', 'user-cam.id']})

## CREATE OUTPUT FILE WITH DYNAMIC NAME DERIVED FROM MAPPING FILE

In [53]:
''.join(mapping_file.split(sep='.')[2:4])

'AQMMentalHealthQuestionsV1'

In [54]:
_output_filename =''.join(mapping_file.split(sep='.')[2:4])
_output_filename

'AQMMentalHealthQuestionsV1'

## REMOVE OUTPUT FILE IF EXISTING

In [55]:
if os.path.exists(merged_dir + _output_filename + '.csv'):
    os.remove(merged_dir + _output_filename + '.csv' )

## WRITE EMPTY CSV FOR MERGE.csv

### TODO: FILENAME MUST BE DYNAMIC

## HARDCODE TEST DATA

In [56]:
test_filename_per_field_dd = {
    'health_informations': ['health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_pressure.second_reading.diastole',
              'health_informations.blood_pressure.second_reading.systole',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
    'health_informations.family_history': ['health_informations.family_history']}

test_filename_per_field_dd = {'profiles': ['profiles.civil_status','profiles.employment.is_employed','profiles.education','profiles.employment.nature','profiles.religion']}
test_filename_per_field_dd = {
        'health_informations': [
                  'health_informations.allergies',
                  'health_informations.blood_pressure.first_reading.diastole',
                  'health_informations.blood_pressure.first_reading.systole',
                  'health_informations.blood_sign',
                  'health_informations.exercise_in_a_week',
                  'health_informations.smoking_habit'],
        'health_informations.family_history': [
                  'health_informations.family_history'],
        'profiles': ['profiles.civil_status'],
        'resident': ['gender','registered_at','user-cam.id']}

## DYNAMIC TEST DATA

In [57]:
test_filename_per_field_dd = filename_per_field_dd

## CREATE OUTPUT FILE WITH HEADERS BASED ON _META AND MAPPING FILE

In [58]:
fields_list = list(test_filename_per_field_dd.values())
flat_fields_list = [item for sublist in fields_list for item in sublist]
all_fields_list  = ['_id','_index_map'] + flat_fields_list

empty_data_df = pd.DataFrame(columns=all_fields_list)
empty_data_df.to_csv(merged_dir + _output_filename + '.csv', encoding='utf-8', mode='w', header=True,index=False)
empty_data_df

,_id,_index_map,mental_healths.cause,mental_healths.experience,organization,registered_at,user-cam.id


## TODO: DYNAMICALLY MERGE DATA FROM DIFFERENT 

In [59]:
fields_list = list(test_filename_per_field_dd.values())
flat_fields_list = [item for sublist in fields_list for item in sublist]
flat_fields_list

['mental_healths.cause',
 'mental_healths.experience',
 'organization',
 'registered_at',
 'user-cam.id']

In [60]:
list(test_filename_per_field_dd.items())

[('mental_healths.cause', ['mental_healths.cause']),
 ('mental_healths.experience', ['mental_healths.experience']),
 ('resident', ['organization', 'registered_at', 'user-cam.id'])]

## TODO: identify here if primitive or list. if list .agg to_list

In [61]:
match_headers_df

,file_name,field_name,field_type
0,mental_healths.cause,mental_healths.cause,list
1,mental_healths.experience,mental_healths.experience,list
2,resident,organization,primitive
3,resident,registered_at,primitive
4,resident,user-cam.id,primitive


In [62]:
filename
match_headers_df[match_headers_df['file_name'] == filename][['field_type','file_name']]

,field_type,file_name
2,primitive,resident
3,primitive,resident
4,primitive,resident


In [63]:
list(match_headers_df[match_headers_df['file_name'] == filename][['field_type','file_name']]['field_type'].unique())

['primitive']

## TODO: optimize here. tmp_df will run out of memory

## TODO: TODO: if index_map = 0, replace with empty

## TODO: Remove repeatinng values in tolist()

In [64]:
family = ["KID", "KID","KID"]
family_1 = list(dict.fromkeys(family))
family_1 

['KID']

In [65]:
def _remove_duplicate_list(item):
    return list(dict.fromkeys(item.tolist()))
    
def _array_to_list(_test_df):
    #return _test_df.groupby('_id').agg(lambda x: list(dict.fromkeys(x.tolist())) )
    #return _test_df.groupby('_id').agg(lambda x: x.tolist() )
    return _test_df.groupby('_id').agg(lambda x: _remove_duplicate_list(x) )

In [66]:
def _flatten_index_map(index):
    _new_index = ''
    
    if index == '0':
        _new_index = '' 
    elif isinstance(index,list):
        _new_index = ''
    else:
        _new_index = index
        
    return _new_index

In [67]:
_tmp_df = pd.DataFrame(columns=['_id','_index_map'])
_tmp_df.to_csv(merged_dir + _output_filename + '.csv',index=False)

for filename, fields in test_filename_per_field_dd.items():
    
    _mandatory_fields = ['_id','_index_map']
    _fields = list(fields + _mandatory_fields)
    
    _field_type = list(match_headers_df[match_headers_df['file_name'] == filename][['field_type','file_name']]['field_type'].unique())[0]
    
    
    _test_df = pd.DataFrame()
    _test_df = pd.read_csv(processed_dir + str(filename) + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
    
    _test_df = _test_df[_fields].replace(np.nan,'',regex=True).reset_index(drop=True)
    
    if _field_type == 'list':
        _test_df = _array_to_list(_test_df)


    _test_df['_index_map'] = _test_df['_index_map'].apply(_flatten_index_map)
    
    print('Start Merging ', filename)
    _tmp_df = _tmp_df.merge(_test_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') )
    #.replace(np.nan,'',regex=True)
    

_tmp_df.to_csv(merged_dir + _output_filename + '.csv',index=False)
_tmp_df.T

Start Merging  mental_healths.cause
Start Merging  mental_healths.experience
Start Merging  resident


,0,1,2,3,4,5,6,7,8,9,...,8144,8145,8146,8147,8148,8149,8150,8151,8152,8153
_id,38ac4495-fad2-4b6d-b31c-077ae092625c,451104d9-dac7-4e26-8dc8-0614cb1d722c,4924a209-c23b-4892-8def-cb1ee2c8409f,58094826-5491-4e0d-a448-ff45c469fc02,9b0944ca-d47f-44b9-bc9d-0c5682325f46,a248236d-48d6-4c6a-ac34-0d3ae7323a49,b79d9289-99f1-4613-85e2-e9e3d9696df2,b9d3adcb-776e-4935-897c-76239f99b054,edebd73a-2b98-4ccd-b6cc-295b7215105a,1405babc-3c7f-484e-9ef9-318cf84d5ebc,...,ffc1d35d-c805-4a37-9aea-dcdeb293046d,ffc2ce26-b83e-43b5-a949-d4680dc11e40,ffc8315c-5e6b-438b-9eac-4960689b488e,ffd21478-98c3-4041-81c4-09012e7b49f6,ffd38085-3e0d-4bd5-a794-9203e7c27bab,ffdc99be-c52f-4302-a2ef-f41067f249fc,ffde3bf1-8515-4841-9ac7-197bf560d3d8,ffe70ad6-b66e-4a24-9251-957ea0f0bc1a,ffe79b96-b393-481f-867d-ac5fd944ce94,ffefcb89-1a0a-4922-815d-3b1c184096c0
mental_healths.cause,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Significant or consistent problems in work an...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
_index_map,,,,,,,,,,,...,,,,,,,,,,
mental_healths.experience,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Significant or consistent problems in work an...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of anxie...,[Extreme or consistent discomfort out of stron...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
organization,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,...,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU
registered_at,03/21/2018 at 03:43:26 pm GMT+08:00,03/21/2018 at 02:45:09 pm GMT+08:00,03/21/2018 at 04:28:16 pm GMT+08:00,03/21/2018 at 05:36:53 pm GMT+08:00,03/21/2018 at 12:52:19 PM GMT+08:00,03/21/2018 at 03:03:08 pm GMT+08:00,03/21/2018 at 03:41:15 pm GMT+08:00,03/21/2018 at 06:12:23 AM GMT+00:00,03/21/2018 at 05:10:48 pm GMT+08:00,04/17/2018 at 10:19:24 AM GMT+08:00,...,04/17/2018 at 08:56:38 AM GMT+08:00,04/16/2018 at 10:58:48 am GMT+08:00,05/21/2018 at 10:16:52 PM GMT-04:00,04/29/2018 at 10:31:40 PM GMT-04:00,05/22/2018 at 09:51:10 AM GMT+08:00,05/16/2018 at 08:53:06 AM GMT+08:00,05/08/2018 at 09:16:17 AM GMT+08:00,05/09/2018 at 11:43:49 AM GMT+08:00,04/16/2018 at 03:21:18 PM GMT+08:00,05/09/2018 at 11:43:02 AM GMT+08:00
user-cam.id,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,particion@gmail.com,...,particion@gmail.com,bongolsanvicente@gmail.com,pescadores@gmail.com,buluangan@gmail.com,torreblancablumentritt@gmail.com,libo-ongonzales@gmail.com,igcocolo@gmail.com,nanga@gmail.com,bongolsanvicente@gmail.com,nanga@gmail.com


## WRITE MERGE DATA TO CSV

In [68]:
#test_group_df = pd.read_csv(merged_dir + _output_filename + '.csv',dtype={'_index_map': str}).replace(np.nan,'',regex=True)
test_group_df = pd.read_csv(merged_dir + _output_filename + '.csv',dtype={'_index_map': str})

In [69]:
test_group_df.T

,0,1,2,3,4,5,6,7,8,9,...,8144,8145,8146,8147,8148,8149,8150,8151,8152,8153
_id,38ac4495-fad2-4b6d-b31c-077ae092625c,451104d9-dac7-4e26-8dc8-0614cb1d722c,4924a209-c23b-4892-8def-cb1ee2c8409f,58094826-5491-4e0d-a448-ff45c469fc02,9b0944ca-d47f-44b9-bc9d-0c5682325f46,a248236d-48d6-4c6a-ac34-0d3ae7323a49,b79d9289-99f1-4613-85e2-e9e3d9696df2,b9d3adcb-776e-4935-897c-76239f99b054,edebd73a-2b98-4ccd-b6cc-295b7215105a,1405babc-3c7f-484e-9ef9-318cf84d5ebc,...,ffc1d35d-c805-4a37-9aea-dcdeb293046d,ffc2ce26-b83e-43b5-a949-d4680dc11e40,ffc8315c-5e6b-438b-9eac-4960689b488e,ffd21478-98c3-4041-81c4-09012e7b49f6,ffd38085-3e0d-4bd5-a794-9203e7c27bab,ffdc99be-c52f-4302-a2ef-f41067f249fc,ffde3bf1-8515-4841-9ac7-197bf560d3d8,ffe70ad6-b66e-4a24-9251-957ea0f0bc1a,ffe79b96-b393-481f-867d-ac5fd944ce94,ffefcb89-1a0a-4922-815d-3b1c184096c0
mental_healths.cause,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Significant or consistent problems in work a...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
_index_map,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mental_healths.experience,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Significant or consistent problems in work a...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of stro...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
organization,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,...,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU,Guimbal RHU
registered_at,03/21/2018 at 03:43:26 pm GMT+08:00,03/21/2018 at 02:45:09 pm GMT+08:00,03/21/2018 at 04:28:16 pm GMT+08:00,03/21/2018 at 05:36:53 pm GMT+08:00,03/21/2018 at 12:52:19 PM GMT+08:00,03/21/2018 at 03:03:08 pm GMT+08:00,03/21/2018 at 03:41:15 pm GMT+08:00,03/21/2018 at 06:12:23 AM GMT+00:00,03/21/2018 at 05:10:48 pm GMT+08:00,04/17/2018 at 10:19:24 AM GMT+08:00,...,04/17/2018 at 08:56:38 AM GMT+08:00,04/16/2018 at 10:58:48 am GMT+08:00,05/21/2018 at 10:16:52 PM GMT-04:00,04/29/2018 at 10:31:40 PM GMT-04:00,05/22/2018 at 09:51:10 AM GMT+08:00,05/16/2018 at 08:53:06 AM GMT+08:00,05/08/2018 at 09:16:17 AM GMT+08:00,05/09/2018 at 11:43:49 AM GMT+08:00,04/16/2018 at 03:21:18 PM GMT+08:00,05/09/2018 at 11:43:02 AM GMT+08:00
user-cam.id,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,guimbalrhucam25@gmail.com,particion@gmail.com,...,particion@gmail.com,bongolsanvicente@gmail.com,pescadores@gmail.com,buluangan@gmail.com,torreblancablumentritt@gmail.com,libo-ongonzales@gmail.com,igcocolo@gmail.com,nanga@gmail.com,bongolsanvicente@gmail.com,nanga@gmail.com


## RENAME THE HEADERS USING MAPPED HEADERS

In [70]:
source_destination_keys_df = pd.DataFrame()
source_destination_keys_df = _mapping_df[['source_key','destination_key']]

In [71]:
new_column_name_dict = dict(zip(source_destination_keys_df['source_key'], source_destination_keys_df['destination_key']))
new_column_name_dict 

{'mental_healths.cause': 'answers.mentalHealthEffect',
 'mental_healths.experience': 'answers.mentalHealthExperience',
 'organization': 'organization',
 'registered_at': 'dateCreated',
 'user-cam.id': 'createdBy',
 '': 'type'}

## GET THE DATA from merged dataframes

In [72]:
_required_data_df = pd.DataFrame()
_required_data_df = test_group_df

## APPLICABLE only if not Resident data

## !!!!++++START TRANSFOMRATION HERE++++!!!!

## DO NECESSARY TRANSFORMATION

In [73]:
_required_data_df.rename(columns = new_column_name_dict,inplace=True )

## APPLICABLE only if NOT Resident data
_required_data_df.rename(columns={'_id': 'profileId'}, inplace=True)
_required_data_df.head(3)

,profileId,answers.mentalHealthEffect,_index_map,answers.mentalHealthExperience,organization,dateCreated,createdBy
0,38ac4495-fad2-4b6d-b31c-077ae092625c,['Extreme or consistent discomfort out of anxi...,NaN,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:43:26 pm GMT+08:00,guimbalrhucam25@gmail.com
1,451104d9-dac7-4e26-8dc8-0614cb1d722c,['Extreme or consistent discomfort out of anxi...,NaN,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 02:45:09 pm GMT+08:00,guimbalrhucam25@gmail.com
2,4924a209-c23b-4892-8def-cb1ee2c8409f,['Extreme or consistent discomfort out of anxi...,NaN,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 04:28:16 pm GMT+08:00,guimbalrhucam25@gmail.com


## GENERATE _id column

In [74]:
_required_data_df['_id'] = _required_data_df.index.to_series().map(lambda x: uuid.uuid4())
_required_data_df['id'] = _required_data_df['_id']


## FILL np.NAN for createdBy, Clean other np.Nan

In [75]:
_required_data_df['createdBy'] = _required_data_df['createdBy'].fillna(method='ffill')
_required_data_df['organization'] = _required_data_df['organization'].fillna(method='ffill')
_required_data_df = _required_data_df.replace(np.nan,'',regex=True)
_required_data_df

,profileId,answers.mentalHealthEffect,_index_map,answers.mentalHealthExperience,organization,dateCreated,createdBy,_id,id
0,38ac4495-fad2-4b6d-b31c-077ae092625c,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:43:26 pm GMT+08:00,guimbalrhucam25@gmail.com,11b0c681-31a6-41ed-9545-f15afe1534d1,11b0c681-31a6-41ed-9545-f15afe1534d1
1,451104d9-dac7-4e26-8dc8-0614cb1d722c,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 02:45:09 pm GMT+08:00,guimbalrhucam25@gmail.com,60196f75-9b89-44e8-b59b-c50d26eda854,60196f75-9b89-44e8-b59b-c50d26eda854
2,4924a209-c23b-4892-8def-cb1ee2c8409f,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 04:28:16 pm GMT+08:00,guimbalrhucam25@gmail.com,f2a62d75-6230-4359-85e1-76b2fdb9954b,f2a62d75-6230-4359-85e1-76b2fdb9954b
3,58094826-5491-4e0d-a448-ff45c469fc02,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 05:36:53 pm GMT+08:00,guimbalrhucam25@gmail.com,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb
4,9b0944ca-d47f-44b9-bc9d-0c5682325f46,['Significant or consistent problems in work a...,,['Significant or consistent problems in work a...,Guimbal RHU,03/21/2018 at 12:52:19 PM GMT+08:00,guimbalrhucam25@gmail.com,7c306f6d-89f6-4039-bb0d-a7350ad940b5,7c306f6d-89f6-4039-bb0d-a7350ad940b5
5,a248236d-48d6-4c6a-ac34-0d3ae7323a49,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:03:08 pm GMT+08:00,guimbalrhucam25@gmail.com,9bbc8927-1526-458d-9c1d-3a2ac8acd45c,9bbc8927-1526-458d-9c1d-3a2ac8acd45c
6,b79d9289-99f1-4613-85e2-e9e3d9696df2,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:41:15 pm GMT+08:00,guimbalrhucam25@gmail.com,41022cc2-b824-4b77-a32e-1a2c90c7ba1e,41022cc2-b824-4b77-a32e-1a2c90c7ba1e
7,b9d3adcb-776e-4935-897c-76239f99b054,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 06:12:23 AM GMT+00:00,guimbalrhucam25@gmail.com,8acee587-766b-4cec-aedd-acbc2e71ba89,8acee587-766b-4cec-aedd-acbc2e71ba89
8,edebd73a-2b98-4ccd-b6cc-295b7215105a,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 05:10:48 pm GMT+08:00,guimbalrhucam25@gmail.com,cae346b0-94ff-47d2-96a2-fe1d8074ed01,cae346b0-94ff-47d2-96a2-fe1d8074ed01
9,1405babc-3c7f-484e-9ef9-318cf84d5ebc,,,['Extreme or consistent discomfort out of stro...,Guimbal RHU,04/17/2018 at 10:19:24 AM GMT+08:00,particion@gmail.com,a7990512-700a-4658-91fe-ee4c32600a92,a7990512-700a-4658-91fe-ee4c32600a92


## CLEAR _INDEX_MAP since each _id has generaed ID

In [76]:
_required_data_df['_index_map'] = ''
_required_data_df

,profileId,answers.mentalHealthEffect,_index_map,answers.mentalHealthExperience,organization,dateCreated,createdBy,_id,id
0,38ac4495-fad2-4b6d-b31c-077ae092625c,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:43:26 pm GMT+08:00,guimbalrhucam25@gmail.com,11b0c681-31a6-41ed-9545-f15afe1534d1,11b0c681-31a6-41ed-9545-f15afe1534d1
1,451104d9-dac7-4e26-8dc8-0614cb1d722c,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 02:45:09 pm GMT+08:00,guimbalrhucam25@gmail.com,60196f75-9b89-44e8-b59b-c50d26eda854,60196f75-9b89-44e8-b59b-c50d26eda854
2,4924a209-c23b-4892-8def-cb1ee2c8409f,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 04:28:16 pm GMT+08:00,guimbalrhucam25@gmail.com,f2a62d75-6230-4359-85e1-76b2fdb9954b,f2a62d75-6230-4359-85e1-76b2fdb9954b
3,58094826-5491-4e0d-a448-ff45c469fc02,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 05:36:53 pm GMT+08:00,guimbalrhucam25@gmail.com,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb
4,9b0944ca-d47f-44b9-bc9d-0c5682325f46,['Significant or consistent problems in work a...,,['Significant or consistent problems in work a...,Guimbal RHU,03/21/2018 at 12:52:19 PM GMT+08:00,guimbalrhucam25@gmail.com,7c306f6d-89f6-4039-bb0d-a7350ad940b5,7c306f6d-89f6-4039-bb0d-a7350ad940b5
5,a248236d-48d6-4c6a-ac34-0d3ae7323a49,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:03:08 pm GMT+08:00,guimbalrhucam25@gmail.com,9bbc8927-1526-458d-9c1d-3a2ac8acd45c,9bbc8927-1526-458d-9c1d-3a2ac8acd45c
6,b79d9289-99f1-4613-85e2-e9e3d9696df2,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:41:15 pm GMT+08:00,guimbalrhucam25@gmail.com,41022cc2-b824-4b77-a32e-1a2c90c7ba1e,41022cc2-b824-4b77-a32e-1a2c90c7ba1e
7,b9d3adcb-776e-4935-897c-76239f99b054,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 06:12:23 AM GMT+00:00,guimbalrhucam25@gmail.com,8acee587-766b-4cec-aedd-acbc2e71ba89,8acee587-766b-4cec-aedd-acbc2e71ba89
8,edebd73a-2b98-4ccd-b6cc-295b7215105a,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 05:10:48 pm GMT+08:00,guimbalrhucam25@gmail.com,cae346b0-94ff-47d2-96a2-fe1d8074ed01,cae346b0-94ff-47d2-96a2-fe1d8074ed01
9,1405babc-3c7f-484e-9ef9-318cf84d5ebc,,,['Extreme or consistent discomfort out of stro...,Guimbal RHU,04/17/2018 at 10:19:24 AM GMT+08:00,particion@gmail.com,a7990512-700a-4658-91fe-ee4c32600a92,a7990512-700a-4658-91fe-ee4c32600a92


## GET THE NEW FIELDS WITH THE DEFAULT VALUES

In [77]:
new_fields_df = _mapping_df[_mapping_df['data_type'] == 'new'][['destination_key', 'default_value']]
new_fields_df

,destination_key,default_value
5,formId,ajWmReH7k2G4mVuvYgYrpX
6,formName,2.6 - AQM Mental Health Questions V1
7,type,profile-related-form


## APPEND THE NEW FIELDS AS COLUMN

In [78]:
for index,row in new_fields_df.iterrows():
    _header = row['destination_key']
    _value = row['default_value']
    
    _required_data_df[_header] = _value

_required_data_df = _required_data_df.reset_index(drop=True)
_required_data_df.head(5)

,profileId,answers.mentalHealthEffect,_index_map,answers.mentalHealthExperience,organization,dateCreated,createdBy,_id,id,formId,formName,type
0,38ac4495-fad2-4b6d-b31c-077ae092625c,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 03:43:26 pm GMT+08:00,guimbalrhucam25@gmail.com,11b0c681-31a6-41ed-9545-f15afe1534d1,11b0c681-31a6-41ed-9545-f15afe1534d1,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,profile-related-form
1,451104d9-dac7-4e26-8dc8-0614cb1d722c,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 02:45:09 pm GMT+08:00,guimbalrhucam25@gmail.com,60196f75-9b89-44e8-b59b-c50d26eda854,60196f75-9b89-44e8-b59b-c50d26eda854,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,profile-related-form
2,4924a209-c23b-4892-8def-cb1ee2c8409f,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 04:28:16 pm GMT+08:00,guimbalrhucam25@gmail.com,f2a62d75-6230-4359-85e1-76b2fdb9954b,f2a62d75-6230-4359-85e1-76b2fdb9954b,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,profile-related-form
3,58094826-5491-4e0d-a448-ff45c469fc02,['Extreme or consistent discomfort out of anxi...,,['Extreme or consistent discomfort out of anxi...,Guimbal RHU,03/21/2018 at 05:36:53 pm GMT+08:00,guimbalrhucam25@gmail.com,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,profile-related-form
4,9b0944ca-d47f-44b9-bc9d-0c5682325f46,['Significant or consistent problems in work a...,,['Significant or consistent problems in work a...,Guimbal RHU,03/21/2018 at 12:52:19 PM GMT+08:00,guimbalrhucam25@gmail.com,7c306f6d-89f6-4039-bb0d-a7350ad940b5,7c306f6d-89f6-4039-bb0d-a7350ad940b5,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,profile-related-form


## SORT COLUMN HEADERS

In [79]:
_sorted_columns = sorted(list(_required_data_df.columns))
_required_data_df = _required_data_df[_sorted_columns]
_required_data_df

,_id,_index_map,answers.mentalHealthEffect,answers.mentalHealthExperience,createdBy,dateCreated,formId,formName,id,organization,profileId,type
0,11b0c681-31a6-41ed-9545-f15afe1534d1,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 03:43:26 pm GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,11b0c681-31a6-41ed-9545-f15afe1534d1,Guimbal RHU,38ac4495-fad2-4b6d-b31c-077ae092625c,profile-related-form
1,60196f75-9b89-44e8-b59b-c50d26eda854,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 02:45:09 pm GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,60196f75-9b89-44e8-b59b-c50d26eda854,Guimbal RHU,451104d9-dac7-4e26-8dc8-0614cb1d722c,profile-related-form
2,f2a62d75-6230-4359-85e1-76b2fdb9954b,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 04:28:16 pm GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,f2a62d75-6230-4359-85e1-76b2fdb9954b,Guimbal RHU,4924a209-c23b-4892-8def-cb1ee2c8409f,profile-related-form
3,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 05:36:53 pm GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,Guimbal RHU,58094826-5491-4e0d-a448-ff45c469fc02,profile-related-form
4,7c306f6d-89f6-4039-bb0d-a7350ad940b5,,['Significant or consistent problems in work a...,['Significant or consistent problems in work a...,guimbalrhucam25@gmail.com,03/21/2018 at 12:52:19 PM GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,7c306f6d-89f6-4039-bb0d-a7350ad940b5,Guimbal RHU,9b0944ca-d47f-44b9-bc9d-0c5682325f46,profile-related-form
5,9bbc8927-1526-458d-9c1d-3a2ac8acd45c,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 03:03:08 pm GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,9bbc8927-1526-458d-9c1d-3a2ac8acd45c,Guimbal RHU,a248236d-48d6-4c6a-ac34-0d3ae7323a49,profile-related-form
6,41022cc2-b824-4b77-a32e-1a2c90c7ba1e,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 03:41:15 pm GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,41022cc2-b824-4b77-a32e-1a2c90c7ba1e,Guimbal RHU,b79d9289-99f1-4613-85e2-e9e3d9696df2,profile-related-form
7,8acee587-766b-4cec-aedd-acbc2e71ba89,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 06:12:23 AM GMT+00:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,8acee587-766b-4cec-aedd-acbc2e71ba89,Guimbal RHU,b9d3adcb-776e-4935-897c-76239f99b054,profile-related-form
8,cae346b0-94ff-47d2-96a2-fe1d8074ed01,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,03/21/2018 at 05:10:48 pm GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,cae346b0-94ff-47d2-96a2-fe1d8074ed01,Guimbal RHU,edebd73a-2b98-4ccd-b6cc-295b7215105a,profile-related-form
9,a7990512-700a-4658-91fe-ee4c32600a92,,,['Extreme or consistent discomfort out of stro...,particion@gmail.com,04/17/2018 at 10:19:24 AM GMT+08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,a7990512-700a-4658-91fe-ee4c32600a92,Guimbal RHU,1405babc-3c7f-484e-9ef9-318cf84d5ebc,profile-related-form


## WRITE TO CSV

In [80]:
_required_data_df.to_csv(merged_dir + _output_filename + '.csv', encoding='utf-8', mode='w', header=True,index=False)

# ===== DO DATA CASTING HERE ====

## GET CSV DATA

In [81]:
_saved_data_df = pd.read_csv(merged_dir + _output_filename + '.csv')

## GET CSV MAPPING

In [82]:
_required_transformation_df = _mapping_df[_mapping_df['data_transformation'] == 'required'].reset_index(drop=True)
_required_transformation_df = _required_transformation_df[['destination_key','destination_type','data_format']]
_required_transformation_df

,destination_key,destination_type,data_format
0,dateCreated,timestamp,timestamp


In [83]:
def _parsed_datetime(datetime):
    _new_datetime = None
    print('parsing datetime: ' , str(datetime) )
    
    
    if isinstance(datetime,str):
        _new_datetime = (parser.parse(datetime).isoformat())
    else: 
        _new_datetime = datetime
       
    
    return _new_datetime

In [84]:
def _typecast_data(data, data_type):
    _cast_data = None
    
    if data_type == 'timestamp':
        _cast_data = _parsed_datetime(data)
    elif data_type == 'boolean':
        _cast_data = bool(lower(data))
    elif data_type == 'integer':
        _cast_data = int(data)
    else:
        _cast_data = str(data)
        
    return _cast_data

In [85]:
for index, row in _required_transformation_df.iterrows():
    print('type casting.. ' + _header)
    
    _header = row['destination_key']
    _type = row['destination_type']
    _format = row['data_format']
    
    _saved_data_df[_header] = _saved_data_df[_header].apply(lambda row: _typecast_data(row, _type))

_saved_data_df = _saved_data_df.replace(np.nan,'',regex=True)
_saved_data_df

type casting.. type
parsing datetime:  03/21/2018 at 03:43:26 pm GMT+08:00
parsing datetime:  03/21/2018 at 02:45:09 pm GMT+08:00
parsing datetime:  03/21/2018 at 04:28:16 pm GMT+08:00
parsing datetime:  03/21/2018 at 05:36:53 pm GMT+08:00
parsing datetime:  03/21/2018 at 12:52:19 PM GMT+08:00
parsing datetime:  03/21/2018 at 03:03:08 pm GMT+08:00
parsing datetime:  03/21/2018 at 03:41:15 pm GMT+08:00
parsing datetime:  03/21/2018 at 06:12:23 AM GMT+00:00
parsing datetime:  03/21/2018 at 05:10:48 pm GMT+08:00
parsing datetime:  04/17/2018 at 10:19:24 AM GMT+08:00
parsing datetime:  03/23/2018 at 03:00:12 PM GMT+08:00
parsing datetime:  03/21/2018 at 02:09:20 PM GMT+08:00
parsing datetime:  05/10/2018 at 09:18:06 PM GMT-04:00
parsing datetime:  04/16/2018 at 03:03:18 pm GMT+08:00
parsing datetime:  05/10/2018 at 08:33:46 PM GMT-04:00
parsing datetime:  04/16/2018 at 07:19:56 AM GMT+00:00
parsing datetime:  04/20/2018 at 09:27:07 AM GMT+08:00
parsing datetime:  05/10/2018 at 09:32:20 PM 

,_id,_index_map,answers.mentalHealthEffect,answers.mentalHealthExperience,createdBy,dateCreated,formId,formName,id,organization,profileId,type
0,11b0c681-31a6-41ed-9545-f15afe1534d1,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T15:43:26-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,11b0c681-31a6-41ed-9545-f15afe1534d1,Guimbal RHU,38ac4495-fad2-4b6d-b31c-077ae092625c,profile-related-form
1,60196f75-9b89-44e8-b59b-c50d26eda854,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T14:45:09-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,60196f75-9b89-44e8-b59b-c50d26eda854,Guimbal RHU,451104d9-dac7-4e26-8dc8-0614cb1d722c,profile-related-form
2,f2a62d75-6230-4359-85e1-76b2fdb9954b,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T16:28:16-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,f2a62d75-6230-4359-85e1-76b2fdb9954b,Guimbal RHU,4924a209-c23b-4892-8def-cb1ee2c8409f,profile-related-form
3,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T17:36:53-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,31f0d2dc-fef3-4c17-9ff5-c7018f570dbb,Guimbal RHU,58094826-5491-4e0d-a448-ff45c469fc02,profile-related-form
4,7c306f6d-89f6-4039-bb0d-a7350ad940b5,,['Significant or consistent problems in work a...,['Significant or consistent problems in work a...,guimbalrhucam25@gmail.com,2018-03-21T12:52:19-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,7c306f6d-89f6-4039-bb0d-a7350ad940b5,Guimbal RHU,9b0944ca-d47f-44b9-bc9d-0c5682325f46,profile-related-form
5,9bbc8927-1526-458d-9c1d-3a2ac8acd45c,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T15:03:08-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,9bbc8927-1526-458d-9c1d-3a2ac8acd45c,Guimbal RHU,a248236d-48d6-4c6a-ac34-0d3ae7323a49,profile-related-form
6,41022cc2-b824-4b77-a32e-1a2c90c7ba1e,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T15:41:15-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,41022cc2-b824-4b77-a32e-1a2c90c7ba1e,Guimbal RHU,b79d9289-99f1-4613-85e2-e9e3d9696df2,profile-related-form
7,8acee587-766b-4cec-aedd-acbc2e71ba89,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T06:12:23+00:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,8acee587-766b-4cec-aedd-acbc2e71ba89,Guimbal RHU,b9d3adcb-776e-4935-897c-76239f99b054,profile-related-form
8,cae346b0-94ff-47d2-96a2-fe1d8074ed01,,['Extreme or consistent discomfort out of anxi...,['Extreme or consistent discomfort out of anxi...,guimbalrhucam25@gmail.com,2018-03-21T17:10:48-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,cae346b0-94ff-47d2-96a2-fe1d8074ed01,Guimbal RHU,edebd73a-2b98-4ccd-b6cc-295b7215105a,profile-related-form
9,a7990512-700a-4658-91fe-ee4c32600a92,,,['Extreme or consistent discomfort out of stro...,particion@gmail.com,2018-04-17T10:19:24-08:00,ajWmReH7k2G4mVuvYgYrpX,2.6 - AQM Mental Health Questions V1,a7990512-700a-4658-91fe-ee4c32600a92,Guimbal RHU,1405babc-3c7f-484e-9ef9-318cf84d5ebc,profile-related-form


## REWRITE CASTED DATA TO CSV FINALIZED


In [86]:
_saved_data_df.to_csv(merged_dir + _output_filename + '.csv', encoding='utf-8', mode='w', header=True,index=False)